In [ ]:
# import sys
# sys.path.append("/home/shuningjin_google_com/maxtext")
# sys.path.append("/home/shuningjin_google_com/maxtext/MaxText")


# import os

# # 1. Get the current working directory (optional, for verification)
# print("Current working directory:", os.getcwd())

# # 2. Define the path to your MaxText directory
# # Replace '/path/to/your/maxtext_directory' with the actual absolute path
# # to your MaxText project directory.
# # Example: If MaxText is in your home folder, it might be:
# # maxtext_path = os.path.expanduser("~/MaxText")
# maxtext_path = "/home/shuningjin_google_com/maxtext"

# # 3. Change the current working directory
# try:
#     os.chdir(maxtext_path)
#     print(f"Successfully changed directory to: {os.getcwd()}")
# except FileNotFoundError:
#     print(f"Error: The directory '{maxtext_path}' was not found.")
# except Exception as e:
#     print(f"An error occurred: {e}")



<module 'MaxText' from '/home/shuningjin/maxtext/MaxText/__init__.py'>

In [36]:
"""
Copyright 2023 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

     https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
"""

# pylint: disable=g-bad-todo, abstract-method, consider-using-with
"""Transforms a "full state" including optimizer state to a bfloat16 "parameter state" without optimizer state.
   This typically used for turning a state output by training.py into a state than can be consumed by decode.py.

   The input "fullstate" is passed in via:
     load_full_state_path.
   The output "parameter state" is output to the checkpoint directory. Additionally it is cast down to bf16.
"""


from importlib import reload
import MaxText
reload(MaxText)


import os.path
from typing import Sequence

from absl import app

from etils import epath

import jax
from jax.sharding import Mesh
from jax import random

from MaxText import checkpointing
reload(checkpointing)
from MaxText import max_logging
from MaxText import max_utils
from MaxText import maxtext_utils
reload(maxtext_utils)
from MaxText import optimizers
from MaxText import pyconfig
from MaxText.common_types import DecoderBlockType
from MaxText.layers import models, quantizations
from MaxText.train import save_checkpoint
from MaxText.utils import gcs_utils
from MaxText.utils import lora_utils

Transformer = models.Transformer


def _possibly_unroll_params(config, training_state, training_state_annotations, mesh):
  """Unroll scanned input layers when force_unroll is set."""
  if not config.scan_layers or not config.force_unroll:
    return

  def unroll_layer_group(num_layers, layer_name="layers"):
    """Helper function to unroll layers (e.g. dense or MoE) into individual layers."""
    layers = training_state.params["params"]["decoder"].get(layer_name, None)
    layers_annotations = training_state_annotations.params["params"]["decoder"].get(layer_name, None)

    if layers is None or layers_annotations is None:
      raise ValueError(f"Missing {layer_name} in training_state or training_state_annotations.")

    def new_pspec(x):
      return jax.sharding.PartitionSpec(*(x[0 : config.param_scan_axis] + x[config.param_scan_axis + 1 :]))

    new_layer_annotation = jax.tree_util.tree_map(new_pspec, layers_annotations)
    new_layer_sharding = jax.tree_util.tree_map(lambda x: jax.sharding.NamedSharding(mesh, x), new_layer_annotation)

    for i in range(num_layers):

      def slice_ith(input_layers):
        return jax.tree_util.tree_map(lambda x: jax.numpy.take(x, i, axis=config.param_scan_axis), input_layers)

      # pylint: disable=not-callable
      new_layer = jax.jit(slice_ith, out_shardings=new_layer_sharding)(layers)

      training_state.params["params"]["decoder"][f"{layer_name}_{i}"] = new_layer
      training_state_annotations.params["params"]["decoder"][f"{layer_name}_{i}"] = new_layer_annotation

    # Remove the original layer collection
    del training_state.params["params"]["decoder"][layer_name]
    del training_state_annotations.params["params"]["decoder"][layer_name]

    jax.tree_util.tree_map(lambda x: x.delete(), layers)

  if config.decoder_block == DecoderBlockType.DEEPSEEK:
    # Unroll dense and MoE layers separately
    unroll_layer_group(config.first_num_dense_layers, layer_name="dense_layers")
    unroll_layer_group(config.num_decoder_layers - config.first_num_dense_layers, layer_name="moe_layers")
  else:
    unroll_layer_group(config.num_decoder_layers, layer_name="layers")


def _read_train_checkpoint(config, checkpoint_manager, mesh):
  """Read training checkpoint at path defined by load_full_state_path."""
  # Model and Optimizer definition
  quant = quantizations.configure_quantization(config)
  model = Transformer(config, mesh, quant)
  rng = random.PRNGKey(0)
  learning_rate_schedule = maxtext_utils.create_learning_rate_schedule(config)
  tx = optimizers.get_optimizer(config, learning_rate_schedule)
  state, state_mesh_notations, _, _ = maxtext_utils.setup_training_state(
      model, None, tx, config, rng, mesh, checkpoint_manager
  )
  num_params = max_utils.calculate_num_params_from_pytree(state.params)
  max_logging.log(f"In input checkpoint Number of model params={num_params/1e9:.3f} billion")
  return state, state_mesh_notations


# def _generate_lora_decode_checkpoints(config, mesh):
#   """Read lora checkpoints checkpoint at path defined by load_full_state_path."""
#   # Model and Optimizer definition
#   quant = quantizations.configure_quantization(config)
#   model = Transformer(config, mesh, quant)
#   rng = random.PRNGKey(0)
#   learning_rate_schedule = maxtext_utils.create_learning_rate_schedule(config)
#   tx = optimizers.get_optimizer(config, learning_rate_schedule)

#   lora_adapters = gcs_utils.gcs_list_directories(config.lora_input_adapters_path)
#   for lora_id in lora_adapters:
#     # Expected lora_checkpoint_dir = <checkpoint_dir>/loras/<lora_id>
#     lora_checkpoint_dir = os.path.join(config.checkpoint_dir, "loras", lora_id, "")

#     lora_adapter_path = os.path.join(config.lora_input_adapters_path, lora_id, "")

#     # Create a checkpoint manager to save decode checkpoint at lora_checkpoint_dir
#     checkpoint_manager = checkpointing.create_orbax_checkpoint_manager(
#         lora_checkpoint_dir,
#         config.enable_checkpointing,
#         config.async_checkpointing,
#         config.checkpoint_period,
#     )

#     lora_config, lora_state, lora_state_annotations = lora_utils.setup_initial_lora_state(
#         model, None, tx, config, rng, mesh, checkpoint_manager, lora_adapter_path
#     )

#     _possibly_unroll_params(config, lora_state, lora_state_annotations, mesh)

#     gcs_utils.write_dict_to_gcs_json(lora_config, os.path.join(lora_checkpoint_dir, "adapter_config.json"))

#     # Save decode state to config's checkpoint directory at step 0
#     _save_decode_checkpoint(config, lora_state, checkpoint_manager)
#     max_logging.log(f"Successfully saved LoRA checkpoint at: {os.path.join(lora_checkpoint_dir, '0', 'items')}")


# def _save_decode_checkpoint(config, state, checkpoint_manager):
#   """Generate checkpoint for decode from the training_state."""
#   decode_state = maxtext_utils.init_decode_state(
#       None, jax.tree_util.tree_map(lambda x: x.astype(jax.numpy.bfloat16), state.params)
#   )
#   if checkpoint_manager is not None:
#     if save_checkpoint(checkpoint_manager, 0, decode_state):
#       max_logging.log(f"saved an decode checkpoint at {config.checkpoint_dir}")
#   checkpoint_manager.wait_until_finished()


def generate_decode_checkpoint(config):
  """
  Generate an decode checkpoint from a given training checkpoint.
  - Training checkpoint is loaded from config.load_full_state_path.
  - Inference checkpoint will be saved at the config's checkpoint directory.
  """

  devices_array = maxtext_utils.create_device_mesh(config)
  mesh = Mesh(devices_array, config.mesh_axes)

  assert config.checkpoint_dir, "checkpoint_dir not configured"
  # Remove any old checkpoint
  path = epath.Path(config.checkpoint_dir)
  if path.exists():
    if jax.process_index() == 0:
      path.rmtree()

  # Create a checkpoint manager to save decode checkpoint at config.checkpoint_dir
  base_checkpoint_dir = config.checkpoint_dir

  # if config.lora_input_adapters_path:
  #   base_checkpoint_dir += "base/"

  checkpoint_manager = checkpointing.create_orbax_checkpoint_manager(
      base_checkpoint_dir,
      config.enable_checkpointing,
      config.async_checkpointing,
      config.checkpoint_period,
  )
  # Read training state from config.load_paramaters_path
  max_logging.log(f"Read training checkpoint from: {config.load_full_state_path}")
  training_state, training_state_annotations = _read_train_checkpoint(config, checkpoint_manager, mesh)
  _possibly_unroll_params(config, training_state, training_state_annotations, mesh)
  return training_state, training_state_annotations
  # assert training_state.opt_state != {}, "missing opt_state in training checkpoint"

  # print(training_state)
  # print_nested_keys(training_state.params)

  # def print_nested_keys(data, prefix=""):
  #   """
  #   Prints nested keys of a dictionary-like structure in a directory-like format.
  #   Args:
  #       data: The dictionary-like structure to traverse.
  #       prefix: The current path prefix.
  #   """
  #   if isinstance(data, dict):
  #     for key, value in data.items():
  #       current_path = f"{prefix}{key}."
  #       print_nested_keys(value, current_path)
  #   else:
  #     print(f"key: {prefix}")
  #     print(f"value shape: {data.shape}")


#   

#   # Save decode state to config's checkpoint directory at step 0
#   max_logging.log(f"Save decode checkpoint at: {base_checkpoint_dir}")
#   _save_decode_checkpoint(config, training_state, checkpoint_manager)
#   max_logging.log(f"Successfully generated decode checkpoint at: {base_checkpoint_dir}0/items")

#   if config.lora_input_adapters_path:
#     _generate_lora_decode_checkpoints(config, mesh)

  # return True


# def main(argv: Sequence[str]) -> None:
#   print(argv)
#   config = pyconfig.initialize(argv)
#   generate_decode_checkpoint(config)


# if __name__ == "__main__":
#   app.run(main)


def print_nested_keys(data, prefix=""):
  """
  Prints nested keys of a dictionary-like structure in a directory-like format.
  Args:
      data: The dictionary-like structure to traverse.
      prefix: The current path prefix.
  """
  if isinstance(data, dict):
    for key, value in data.items():
      current_path = f"{prefix}{key}."
      print_nested_keys(value, current_path)
  else:
    print(f"key: {prefix}")
    print(f"value shape: {data.shape}")


## scanned param

In [5]:
cmd="python MaxText/configs/base.yml model_name=llama2-7b \
base_output_directory=gs://runner-maxtext-logs run_name=test3 \
load_parameters_path=gs://shuningjin-multipod-dev/llama2-7b/2025-06-22/scanned/0/items scan_layers=true force_unroll=false \
async_checkpointing=false skip_jax_distributed_system=true"
argv = cmd.split()
config = pyconfig.initialize(argv)

Updating keys from env and command line: ['run_name', 'model_name', 'load_parameters_path', 'async_checkpointing', 'force_unroll', 'scan_layers', 'base_output_directory', 'skip_jax_distributed_system']
Running Model: llama2-7b
Updating following parameters in config

base_emb_dim: 4096
base_num_query_heads: 32
base_num_kv_heads: 32
base_mlp_dim: 11008
base_num_decoder_layers: 32
head_dim: 128
mlp_activations: ['silu', 'linear']
vocab_size: 32000
enable_dropout: False
logits_via_embedding: False
normalization_layer_epsilon: 1e-05
decoder_block: llama2
logical_axis_rules: [['norm', 'fsdp']]
Updating keys from model: ['base_emb_dim', 'base_num_query_heads', 'base_num_kv_heads', 'base_mlp_dim', 'base_num_decoder_layers', 'head_dim', 'mlp_activations', 'vocab_size', 'enable_dropout', 'logits_via_embedding', 'normalization_layer_epsilon', 'decoder_block', 'logical_axis_rules']
Skipping jax distributed system due to skip_jax_distributed_system=True flag.
Not using emergency checkpoint, ignori

Failed to find host bounds for accelerator type: WARNING: could not determine TPU accelerator type, please set env var `TPU_ACCELERATOR_TYPE` manually, otherwise libtpu.so may not properly initialize.
E0000 00:00:1751012391.182729 1106180 common_lib.cc:528] INVALID_ARGUMENT: Error: unexpected worker hostname 'WARNING: could not determine TPU worker hostnames or IP addresses' from env var TPU_WORKER_HOSTNAMES. Expecting a valid hostname or IP address without port number. (Full TPU workers' addr string: WARNING: could not determine TPU worker hostnames or IP addresses, please set env var `TPU_WORKER_HOSTNAMES` manually, otherwise libtpu.so may not properly initialize.)
=== Source Location Trace: === 
learning/45eac/tfrc/runtime/libtpu_init_utils.cc:285


Config param prefix_caching_dram_byte: 100000000000
Config param prefix_caching_hbm_byte: 10000000000
Config param profile_cleanly: True
Config param profile_periodically_period: -1
Config param profiler: 
Config param profiler_steps: 5
Config param projector_dropout_for_vit: 0.0
Config param projector_input_dim_for_vit: 4096
Config param projector_output_dim_for_vit: 4096
Config param prometheus_port: 0
Config param prompt: I love to
Config param q_lora_rank: 0
Config param qk_nope_head_dim: 128
Config param qk_rope_head_dim: 64
Config param qkv_proj: remat
Config param quant_cfg_path: 
Config param quantization: 
Config param quantization_local_shard_count: 1
Config param quantize_kvcache: False
Config param query_proj: remat
Config param ragged_block_size: 256
Config param record_internal_nn_metrics: 0
Config param remat_policy: full
Config param remat_policy_for_vit: minimal
Config param replicate_quant_scale: False
Config param replicator_backup_interval_minutes: 0
Config param re

In [6]:
training_state, training_state_annotations = generate_decode_checkpoint(config)

Num_devices: 1, shape (1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)
Creating checkpoint manager with ocdbt=True and zarr3=True
Checkpoint manager created!
Read training checkpoint from: 
checkpoint manager exists so trying to load this run's existing checkpoint
restoring params from gs://shuningjin-multipod-dev/llama2-7b/2025-06-22/scanned/0/items
Creating checkpoint manager with ocdbt=True and zarr3=True
{'params': {'decoder': {'decoder_norm': {'scale': ArrayRestoreArgs(restore_type=<class 'jax.Array'>, dtype=dtype('float32'), mesh=None, mesh_axes=None, sharding=NamedSharding(mesh=Mesh('data': 1, 'stage': 1, 'fsdp': 1, 'fsdp_transpose': 1, 'sequence': 1, 'context': 1, 'context_autoregressive': 1, 'tensor': 1, 'tensor_transpose': 1, 'tensor_sequence': 1, 'expert': 1, 'autoregressive': 1, axis_types=(Auto, Auto, Auto, Auto, Auto, Auto, Auto, Auto, Auto, Auto, Auto, Auto)), spec=PartitionSpec('fsdp',), memory_kind=unpinned_host), global_shape=(4096,), shape=(4096,), strict=True)}, 'layers': {'mlp

I0627 08:19:54.038715 1107048 google_auth_provider.cc:181] Running on GCE, using service account 770040921623-compute@developer.gserviceaccount.com


In input checkpoint Number of model params=6.738 billion


In [7]:
print(training_state)

TrainState(step=Array(0, dtype=int32, weak_type=True), apply_fn=<bound method Module.apply of Transformer(
    # attributes
    config = <MaxText.pyconfig.HyperParameters object at 0x7f7af17ca590>
    mesh = Mesh(device_ids=array([[[[[[[[[[[[0]]]]]]]]]]]]), axis_names=('data', 'stage', 'fsdp', 'fsdp_transpose', 'sequence', 'context', 'context_autoregressive', 'tensor', 'tensor_transpose', 'tensor_sequence', 'expert', 'autoregressive'), axis_types=(Auto, Auto, Auto, Auto, Auto, Auto, Auto, Auto, Auto, Auto, Auto, Auto))
    quant = None
)>, params={'params': {'decoder': {'decoder_norm': {'scale': Array([1.8671875, 1.8671875, 1.8046875, ..., 1.71875  , 1.828125 ,
       1.6015625], dtype=float32)}, 'layers': {'mlp': {'wi_0': {'kernel': Array([[[ 1.57470703e-02, -2.16674805e-03,  6.83593750e-03, ...,
          1.41263008e-05,  2.66113281e-02, -5.98907471e-04],
        [ 2.91748047e-02, -2.80761719e-02, -6.07299805e-03, ...,
         -4.00390625e-02, -1.61132812e-02, -2.49023438e-02],
    

In [8]:
print(training_state.params)

{'params': {'decoder': {'decoder_norm': {'scale': Array([1.8671875, 1.8671875, 1.8046875, ..., 1.71875  , 1.828125 ,
       1.6015625], dtype=float32)}, 'layers': {'mlp': {'wi_0': {'kernel': Array([[[ 1.57470703e-02, -2.16674805e-03,  6.83593750e-03, ...,
          1.41263008e-05,  2.66113281e-02, -5.98907471e-04],
        [ 2.91748047e-02, -2.80761719e-02, -6.07299805e-03, ...,
         -4.00390625e-02, -1.61132812e-02, -2.49023438e-02],
        [ 8.60595703e-03, -2.27928162e-04,  1.76239014e-03, ...,
          2.96630859e-02, -1.24511719e-02, -1.60980225e-03],
        ...,
        [ 8.97216797e-03,  7.41577148e-03,  3.80859375e-02, ...,
         -1.44042969e-02,  1.32446289e-02,  2.12097168e-03],
        [-3.34472656e-02, -1.91650391e-02, -2.18505859e-02, ...,
         -2.44140625e-02,  3.14941406e-02, -8.54492188e-03],
        [ 2.36511230e-03, -7.27539062e-02,  1.79443359e-02, ...,
         -1.50756836e-02,  3.36914062e-02,  2.35595703e-02]],

       [[ 1.70898438e-02, -6.01196289e

In [9]:
training_state_annotations.params

{'params': {'decoder': {'decoder_norm': {'scale': PartitionSpec('fsdp',)},
   'layers': {'mlp': {'wi_0': {'kernel': PartitionSpec(('fsdp', 'sequence', 'tensor_transpose', 'context', 'expert'), 'stage', ('fsdp_transpose', 'tensor', 'tensor_sequence', 'autoregressive'))},
     'wi_1': {'kernel': PartitionSpec(('fsdp', 'sequence', 'tensor_transpose', 'context', 'expert'), 'stage', ('fsdp_transpose', 'tensor', 'tensor_sequence', 'autoregressive'))},
     'wo': {'kernel': PartitionSpec(('fsdp_transpose', 'tensor', 'tensor_sequence', 'autoregressive'), 'stage', ('fsdp', 'sequence', 'tensor_transpose', 'context', 'expert'))}},
    'post_self_attention_layer_norm': {'scale': PartitionSpec('fsdp', 'stage')},
    'pre_self_attention_layer_norm': {'scale': PartitionSpec('fsdp', 'stage')},
    'self_attention': {'key': {'kernel': PartitionSpec(('fsdp', 'fsdp_transpose', 'sequence', 'context', 'expert'), 'stage', ('tensor', 'tensor_transpose', 'tensor_sequence', 'autoregressive'), None)},
     'out

In [10]:
print_nested_keys(training_state.params)

key: params.decoder.decoder_norm.scale.
value shape: (4096,)
key: params.decoder.layers.mlp.wi_0.kernel.
value shape: (4096, 32, 11008)
key: params.decoder.layers.mlp.wi_1.kernel.
value shape: (4096, 32, 11008)
key: params.decoder.layers.mlp.wo.kernel.
value shape: (11008, 32, 4096)
key: params.decoder.layers.post_self_attention_layer_norm.scale.
value shape: (4096, 32)
key: params.decoder.layers.pre_self_attention_layer_norm.scale.
value shape: (4096, 32)
key: params.decoder.layers.self_attention.key.kernel.
value shape: (4096, 32, 32, 128)
key: params.decoder.layers.self_attention.out.kernel.
value shape: (32, 32, 128, 4096)
key: params.decoder.layers.self_attention.query.kernel.
value shape: (4096, 32, 32, 128)
key: params.decoder.layers.self_attention.value.kernel.
value shape: (4096, 32, 32, 128)
key: params.decoder.logits_dense.kernel.
value shape: (4096, 32000)
key: params.token_embedder.embedding.
value shape: (32000, 4096)


## scanned, force unroll

In [11]:
cmd="python MaxText/configs/base.yml model_name=llama2-7b \
base_output_directory=gs://runner-maxtext-logs run_name=test3 \
load_parameters_path=gs://shuningjin-multipod-dev/llama2-7b/2025-06-22/scanned/0/items scan_layers=true force_unroll=true \
async_checkpointing=false skip_jax_distributed_system=true"
argv = cmd.split()
config = pyconfig.initialize(argv)

Updating keys from env and command line: ['run_name', 'model_name', 'load_parameters_path', 'async_checkpointing', 'force_unroll', 'scan_layers', 'base_output_directory', 'skip_jax_distributed_system']
Running Model: llama2-7b
Updating following parameters in config

base_emb_dim: 4096
base_num_query_heads: 32
base_num_kv_heads: 32
base_mlp_dim: 11008
base_num_decoder_layers: 32
head_dim: 128
mlp_activations: ['silu', 'linear']
vocab_size: 32000
enable_dropout: False
logits_via_embedding: False
normalization_layer_epsilon: 1e-05
decoder_block: llama2
logical_axis_rules: [['norm', 'fsdp']]
Updating keys from model: ['base_emb_dim', 'base_num_query_heads', 'base_num_kv_heads', 'base_mlp_dim', 'base_num_decoder_layers', 'head_dim', 'mlp_activations', 'vocab_size', 'enable_dropout', 'logits_via_embedding', 'normalization_layer_epsilon', 'decoder_block', 'logical_axis_rules']
Skipping jax distributed system due to skip_jax_distributed_system=True flag.
Not using emergency checkpoint, ignori

In [12]:
training_state2, training_state_annotations2 = generate_decode_checkpoint(config)

Num_devices: 1, shape (1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)
Creating checkpoint manager with ocdbt=True and zarr3=True
Checkpoint manager created!
Read training checkpoint from: 
checkpoint manager exists so trying to load this run's existing checkpoint
restoring params from gs://shuningjin-multipod-dev/llama2-7b/2025-06-22/scanned/0/items
Creating checkpoint manager with ocdbt=True and zarr3=True
{'params': {'decoder': {'decoder_norm': {'scale': ArrayRestoreArgs(restore_type=<class 'jax.Array'>, dtype=dtype('float32'), mesh=None, mesh_axes=None, sharding=NamedSharding(mesh=Mesh('data': 1, 'stage': 1, 'fsdp': 1, 'fsdp_transpose': 1, 'sequence': 1, 'context': 1, 'context_autoregressive': 1, 'tensor': 1, 'tensor_transpose': 1, 'tensor_sequence': 1, 'expert': 1, 'autoregressive': 1, axis_types=(Auto, Auto, Auto, Auto, Auto, Auto, Auto, Auto, Auto, Auto, Auto, Auto)), spec=PartitionSpec('fsdp',), memory_kind=unpinned_host), global_shape=(4096,), shape=(4096,), strict=True)}, 'layers': {'mlp

2025-06-27 08:21:37.342180: E external/xla/xla/service/slow_operation_alarm.cc:73] Constant folding an instruction is taking > 1s:

  %compare.13 = pred[1]{0} compare(%constant.298, %constant.33), direction=GE, metadata={op_name="jit(slice_ith)/jit(main)/jit(_take)/ge" source_file="/var/tmp/ipykernel_1106180/2432396246.py" source_line=74}

This isn't necessarily a bug; constant-folding is inherently a trade-off between compilation time and speed at runtime. XLA has some guards that attempt to keep constant folding from taking too long, but fundamentally you'll always be able to come up with an input program that takes a long time.

If you'd like to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
2025-06-27 08:21:37.531012: E external/xla/xla/service/slow_operation_alarm.cc:140] The operation took 1.191921684s
Constant folding an instruction is taking > 1s:

  %compare.13 = pred[1]{0} compare(%constant.298, %constant.33), direction=GE, metadata={op_n

In [13]:
import json
json.dumps(training_state_annotations2.params)

TypeError: Object of type PartitionSpec is not JSON serializable

In [14]:
print(training_state_annotations2.params)

{'params': {'decoder': {'decoder_norm': {'scale': PartitionSpec('fsdp',)}, 'logits_dense': {'kernel': PartitionSpec(('fsdp', 'fsdp_transpose', 'sequence', 'context', 'expert'), ('tensor', 'tensor_transpose', 'tensor_sequence', 'autoregressive'))}, 'layers_0': {'mlp': {'wi_0': {'kernel': PartitionSpec(('fsdp', 'sequence', 'tensor_transpose', 'context', 'expert'), ('fsdp_transpose', 'tensor', 'tensor_sequence', 'autoregressive'))}, 'wi_1': {'kernel': PartitionSpec(('fsdp', 'sequence', 'tensor_transpose', 'context', 'expert'), ('fsdp_transpose', 'tensor', 'tensor_sequence', 'autoregressive'))}, 'wo': {'kernel': PartitionSpec(('fsdp_transpose', 'tensor', 'tensor_sequence', 'autoregressive'), ('fsdp', 'sequence', 'tensor_transpose', 'context', 'expert'))}}, 'post_self_attention_layer_norm': {'scale': PartitionSpec('fsdp',)}, 'pre_self_attention_layer_norm': {'scale': PartitionSpec('fsdp',)}, 'self_attention': {'key': {'kernel': PartitionSpec(('fsdp', 'fsdp_transpose', 'sequence', 'context',

In [15]:
print_nested_keys(training_state2.params)

key: params.decoder.decoder_norm.scale.
value shape: (4096,)
key: params.decoder.logits_dense.kernel.
value shape: (4096, 32000)
key: params.decoder.layers_0.mlp.wi_0.kernel.
value shape: (4096, 11008)
key: params.decoder.layers_0.mlp.wi_1.kernel.
value shape: (4096, 11008)
key: params.decoder.layers_0.mlp.wo.kernel.
value shape: (11008, 4096)
key: params.decoder.layers_0.post_self_attention_layer_norm.scale.
value shape: (4096,)
key: params.decoder.layers_0.pre_self_attention_layer_norm.scale.
value shape: (4096,)
key: params.decoder.layers_0.self_attention.key.kernel.
value shape: (4096, 32, 128)
key: params.decoder.layers_0.self_attention.out.kernel.
value shape: (32, 128, 4096)
key: params.decoder.layers_0.self_attention.query.kernel.
value shape: (4096, 32, 128)
key: params.decoder.layers_0.self_attention.value.kernel.
value shape: (4096, 32, 128)
key: params.decoder.layers_1.mlp.wi_0.kernel.
value shape: (4096, 11008)
key: params.decoder.layers_1.mlp.wi_1.kernel.
value shape: (40

## checkpoint1

In [16]:
cmd="python MaxText/configs/base.yml model_name=llama4-17b-16e \
base_output_directory=gs://runner-maxtext-logs run_name=test3 \
load_parameters_path=gs://jacobplatin/llama4/v5-scout-instruct-bf16-final-scanned/0/items scan_layers=true force_unroll=false \
async_checkpointing=false skip_jax_distributed_system=true"
argv = cmd.split()
config = pyconfig.initialize(argv)

Updating keys from env and command line: ['run_name', 'model_name', 'load_parameters_path', 'async_checkpointing', 'force_unroll', 'scan_layers', 'base_output_directory', 'skip_jax_distributed_system']
Running Model: llama4-17b-16e
Updating following parameters in config

decoder_block: llama4
mlp_activations: ['silu', 'linear']
enable_dropout: False
logits_via_embedding: False
tokenizer_type: huggingface
base_emb_dim: 5120
base_num_decoder_layers: 48
base_num_query_heads: 40
base_num_kv_heads: 8
base_mlp_dim: 16384
base_moe_mlp_dim: 8192
vocab_size: 202048
normalization_layer_epsilon: 1e-05
rope_max_timescale: 500000
rope_type: llama3.1
num_experts: 16
shared_experts: 1
num_experts_per_tok: 1
use_qk_norm: True
nope_layer_interval: 4
interleave_moe_layer_step: 1
inhomogeneous_layer_cycle_interval: 4
temperature_tuning: True
chunk_attn_window_size: 8192
image_size_for_vit: 336
Updating keys from model: ['decoder_block', 'mlp_activations', 'enable_dropout', 'logits_via_embedding', 'token

In [17]:
training_state3, training_state_annotations3 = generate_decode_checkpoint(config)

Num_devices: 1, shape (1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)
Creating checkpoint manager with ocdbt=True and zarr3=True
Checkpoint manager created!
Read training checkpoint from: 
checkpoint manager exists so trying to load this run's existing checkpoint
restoring params from gs://jacobplatin/llama4/v5-scout-instruct-bf16-final-scanned/0/items
Creating checkpoint manager with ocdbt=True and zarr3=True
{'params': {'decoder': {'decoder_norm': {'scale': ArrayRestoreArgs(restore_type=<class 'jax.Array'>, dtype=dtype('float32'), mesh=None, mesh_axes=None, sharding=NamedSharding(mesh=Mesh('data': 1, 'stage': 1, 'fsdp': 1, 'fsdp_transpose': 1, 'sequence': 1, 'context': 1, 'context_autoregressive': 1, 'tensor': 1, 'tensor_transpose': 1, 'tensor_sequence': 1, 'expert': 1, 'autoregressive': 1, axis_types=(Auto, Auto, Auto, Auto, Auto, Auto, Auto, Auto, Auto, Auto, Auto, Auto)), spec=PartitionSpec(('tensor', 'tensor_transpose', 'tensor_sequence'),), memory_kind=unpinned_host), global_shape=(5120,),

ValueError: Missing 4 keys in structure path ('params', 'params', 'decoder', 'layers'), including: ['layers_0', 'layers_1', 'layers_2', 'layers_3']

In [ ]:
training_state_annotations3.params

In [ ]:
print_nested_keys(training_state3.params)

## checkpoint 2

In [37]:
cmd="python MaxText/configs/base.yml model_name=llama4-17b-16e \
base_output_directory=gs://runner-maxtext-logs run_name=test3 \
load_parameters_path=gs://shuningjin-multipod-dev/llama4-17b-16e/conversion/meta-scanned/0/items scan_layers=true force_unroll=false \
async_checkpointing=false skip_jax_distributed_system=true"
argv = cmd.split()
config = pyconfig.initialize(argv)

Updating keys from env and command line: ['run_name', 'model_name', 'load_parameters_path', 'async_checkpointing', 'force_unroll', 'scan_layers', 'base_output_directory', 'skip_jax_distributed_system']
Running Model: llama4-17b-16e
Updating following parameters in config

decoder_block: llama4
mlp_activations: ['silu', 'linear']
enable_dropout: False
logits_via_embedding: False
tokenizer_type: huggingface
base_emb_dim: 5120
base_num_decoder_layers: 48
base_num_query_heads: 40
base_num_kv_heads: 8
base_mlp_dim: 16384
base_moe_mlp_dim: 8192
vocab_size: 202048
normalization_layer_epsilon: 1e-05
rope_max_timescale: 500000
rope_type: llama3.1
num_experts: 16
shared_experts: 1
num_experts_per_tok: 1
use_qk_norm: True
nope_layer_interval: 4
interleave_moe_layer_step: 1
inhomogeneous_layer_cycle_interval: 4
temperature_tuning: True
chunk_attn_window_size: 8192
image_size_for_vit: 336
Updating keys from model: ['decoder_block', 'mlp_activations', 'enable_dropout', 'logits_via_embedding', 'token

In [ ]:
training_state4, training_state_annotations4 = generate_decode_checkpoint(config)

Num_devices: 1, shape (1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)
Creating checkpoint manager with ocdbt=True and zarr3=True
Checkpoint manager created!
Read training checkpoint from: 


In [ ]:
print_nested_keys(training_state4.params)

## checkpoint 3

In [20]:
cmd="python MaxText/configs/base.yml model_name=llama4-17b-16e \
base_output_directory=gs://runner-maxtext-logs run_name=test3 \
load_parameters_path=gs://shuningjin-multipod-dev/scout_pretrain/shuning-llama4-2025-06-22-23-49-54/checkpoints/0/items scan_layers=true force_unroll=false \
async_checkpointing=false skip_jax_distributed_system=true"
argv = cmd.split()
config = pyconfig.initialize(argv)

Updating keys from env and command line: ['run_name', 'model_name', 'load_parameters_path', 'async_checkpointing', 'force_unroll', 'scan_layers', 'base_output_directory', 'skip_jax_distributed_system']
Running Model: llama4-17b-16e
Updating following parameters in config

decoder_block: llama4
mlp_activations: ['silu', 'linear']
enable_dropout: False
logits_via_embedding: False
tokenizer_type: huggingface
base_emb_dim: 5120
base_num_decoder_layers: 48
base_num_query_heads: 40
base_num_kv_heads: 8
base_mlp_dim: 16384
base_moe_mlp_dim: 8192
vocab_size: 202048
normalization_layer_epsilon: 1e-05
rope_max_timescale: 500000
rope_type: llama3.1
num_experts: 16
shared_experts: 1
num_experts_per_tok: 1
use_qk_norm: True
nope_layer_interval: 4
interleave_moe_layer_step: 1
inhomogeneous_layer_cycle_interval: 4
temperature_tuning: True
chunk_attn_window_size: 8192
image_size_for_vit: 336
Updating keys from model: ['decoder_block', 'mlp_activations', 'enable_dropout', 'logits_via_embedding', 'token

In [ ]:
training_state5, training_state_annotations5 = generate_decode_checkpoint(config)

Num_devices: 1, shape (1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)
Creating checkpoint manager with ocdbt=True and zarr3=True
Checkpoint manager created!
Read training checkpoint from: 
checkpoint manager exists so trying to load this run's existing checkpoint
restoring params from gs://shuningjin-multipod-dev/scout_pretrain/shuning-llama4-2025-06-22-23-49-54/checkpoints/0/items
Creating checkpoint manager with ocdbt=True and zarr3=True
{'params': {'decoder': {'decoder_norm': {'scale': ArrayRestoreArgs(restore_type=<class 'jax.Array'>, dtype=dtype('float32'), mesh=None, mesh_axes=None, sharding=NamedSharding(mesh=Mesh('data': 1, 'stage': 1, 'fsdp': 1, 'fsdp_transpose': 1, 'sequence': 1, 'context': 1, 'context_autoregressive': 1, 'tensor': 1, 'tensor_transpose': 1, 'tensor_sequence': 1, 'expert': 1, 'autoregressive': 1, axis_types=(Auto, Auto, Auto, Auto, Auto, Auto, Auto, Auto, Auto, Auto, Auto, Auto)), spec=PartitionSpec(('tensor', 'tensor_transpose', 'tensor_sequence'),), memory_kind=unpinne

In [23]:
print_nested_keys(training_state5.params)

key: params.decoder.decoder_norm.scale.
value shape: (5120,)
key: params.decoder.layers.layers_0.Llama4MoEBlock_0.MoeBlock_0.gate.kernel.
value shape: (5120, 12, 16)
key: params.decoder.layers.layers_0.Llama4MoEBlock_0.MoeBlock_0.wi_0.
value shape: (16, 12, 5120, 8192)
key: params.decoder.layers.layers_0.Llama4MoEBlock_0.MoeBlock_0.wi_1.
value shape: (16, 12, 5120, 8192)
key: params.decoder.layers.layers_0.Llama4MoEBlock_0.MoeBlock_0.wo.
value shape: (16, 12, 8192, 5120)
key: params.decoder.layers.layers_0.Llama4MoEBlock_0.shared_experts.wi_0.kernel.
value shape: (5120, 12, 8192)
key: params.decoder.layers.layers_0.Llama4MoEBlock_0.shared_experts.wi_1.kernel.
value shape: (5120, 12, 8192)
key: params.decoder.layers.layers_0.Llama4MoEBlock_0.shared_experts.wo.kernel.
value shape: (8192, 12, 5120)
key: params.decoder.layers.layers_0.post_self_attention_layer_norm.scale.
value shape: (5120, 12)
key: params.decoder.layers.layers_0.pre_self_attention_layer_norm.scale.
value shape: (5120, 12

In [25]:
training_state_annotations5.params

{'params': {'decoder': {'decoder_norm': {'scale': PartitionSpec(('tensor', 'tensor_transpose', 'tensor_sequence'),)},
   'layers': {'layers_0': {'Llama4MoEBlock_0': {'MoeBlock_0': {'gate': {'kernel': PartitionSpec(('fsdp', 'fsdp_transpose', 'sequence', 'tensor_transpose', 'context', 'expert'), 'stage', None)},
       'wi_0': PartitionSpec('expert', 'stage', ('fsdp', 'sequence', 'tensor_transpose', 'context'), ('fsdp_transpose', 'tensor', 'tensor_sequence', 'autoregressive')),
       'wi_1': PartitionSpec('expert', 'stage', ('fsdp', 'sequence', 'tensor_transpose', 'context'), ('fsdp_transpose', 'tensor', 'tensor_sequence', 'autoregressive')),
       'wo': PartitionSpec('expert', 'stage', ('fsdp_transpose', 'tensor', 'tensor_sequence', 'autoregressive'), ('fsdp', 'sequence', 'tensor_transpose', 'context'))},
      'shared_experts': {'wi_0': {'kernel': PartitionSpec(('fsdp', 'sequence', 'tensor_transpose', 'context', 'expert'), 'stage', ('fsdp_transpose', 'tensor', 'tensor_sequence', 'aut